In [1]:

# Imports
import csv
import re
from pathlib import Path

import numpy as np
import pandas as pd

In [2]:
# Project paths

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()

ROOT = PROJECT_ROOT / "data" / "raw" / "vicon_csv"
RESULTS_DIR = PROJECT_ROOT / "results"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print("Project root:", PROJECT_ROOT)
print("Vicon CSV root:", ROOT)
print("ROOT exists:", ROOT.exists())
print("Results dir:", RESULTS_DIR.resolve())

Project root: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2
Vicon CSV root: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2/data/raw/vicon_csv
ROOT exists: True
Results dir: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2/results
